In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import feather
import matplotlib as mpl
mpl.use('nbAgg')
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import label_binarize
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math
from imblearn.combine import SMOTETomek
from sklearn.neural_network import MLPClassifier

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
pingfInfoFilePath = '/content/drive/My Drive/Colab Notebooks/dv_personas_190411_previo_linear_regression'
dv_personas_190411_model = pd.read_feather(pingfInfoFilePath,columns=None, use_threads=True)
dv_personas_190411_model.head()

,sexo,tipo_acceso,tipo_erasmus,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,Titulacion_is_MAESTRO: AUDICIÓN Y LENGUAJE,Titulacion_is_LICENCIADO EN DERECHO,Titulacion_is_MAESTRO: EDUCACIÓN PRIMARIA,Titulacion_is_MAESTRO: EDUCACIÓN INFANTIL,Titulacion_is_DIPLOMADO EN TURISMO,"Titulacion_is_INGENIERÍA TÉCNICA EN TELECOMUNICACIONES,ESPECIALIDAD SONIDO E IMAGEN",Titulacion_is_LICENCIADO EN CIENCIA Y TECNOLOGÍA DE LOS ALIMENTOS,Titulacion_is_LICENCIADO EN CIENCIAS AMBIENTALES,Titulacion_is_LICENCIADO EN CIENCIAS DE LA ACTIVIDAD FÍSICA Y DEL DEPORTE,Titulacion_is_LICENCIADO EN FILOLOGÍA HISPÁNICA,Titulacion_is_GRADO EN EDUCACIÓN SOCIAL,Titulacion_is_LICENCIADO EN MATEMÁTICAS,Titulacion_is_LICENCIADO EN FILOLOGÍA FRANCESA,Titulacion_is_LICENCIADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_GRADO EN DERECHO,Titulacion_is_LICENCIADO EN MEDICINA,Titulacion_is_LICENCIADO EN ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_LICENCIADO EN BIOLOGÍA,Titulacion_is_INGENIERO TÉCNICO EN DISEÑO INDUSTRIAL,Titulacion_is_INGENIERO TÉCNICO EN INFORMÁTICA DE SISTEMAS,Titulacion_is_GRADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN RELACIONES LABORALES,Titulacion_is_LICENCIADO EN ANTROPOLOGÍA SOCIAL Y CULTURAL,Titulacion_is_LICENCIADO EN HISTORIA,Titulacion_is_GRADO EN COMUNICACIÓN AUDIOVISUAL,...,Titulacion_is_MÁSTER UNIVERSITARIO EN MARKETING E INVESTIGACIÓN DE MERCADOS,Titulacion_is_MÁSTER UNIV. FORMACIÓN EN PORTUGUÉS PARA PROF. ENSEÑANZA PRIM. Y SECUNDARIA,Titulacion_is_MÁSTER UNIVERSITARIO EN BIOTECNOLOGÍA AVANZADA,Titulacion_is_MÁSTER U. INVESTIGACIÓN ENSEÑANZA Y APRENDIZAJE DE LAS CC. EXPE SOCIA Y MAT,Titulacion_is_MÁSTER UNIVERSITARIO EN GESTIÓN DE LA INFORMACIÓN DIGITAL,Titulacion_is_GRADO EN INGENIERÍA QUÍMICA INDUSTRIAL,Titulacion_is_MÁSTER UNIVERSITARIO EN ENSEÑANZA BILINGÜE PARA LA EDUCACIÓN PRIMARIA Y SEC,"Titulacion_is_MÁSTER UNIV. INVESTIGACIÓN EN ECONOMÍA, GESTIÓN Y COMERCIO INTERNACIONAL","Titulacion_is_MÁSTER U. EN CONTAMINACIÓN AMBIENTAL: PREVENCIÓN, VIGILANCIA Y CORRECCIÓN",Titulacion_is_MÁSTER UNIVERSITARIO EN DIRECCIÓN TIC,Titulacion_is_MÁSTER UNIVERSITARIO EN GERONTOLOGÍA,Titulacion_is_MÁSTER UNIVERSITARIO EN ANTROPOLOGÍA SOCIAL,Titulacion_is_MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INGENIERÍA INDUSTRIAL,Titulacion_is_MÁSTER UNIVERSITARIO EN INICIACIÓN Y RENDIMIENTO EN EL DEPORTE,Titulacion_is_MÁSTER UNIVERSITARIO EN PROMOCIÓN DE LA SALUD MEDIANTE LA ACTIVIDAD FÍSICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INGENIERÍA AGRONÓMICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INGENIERÍA BIOMÉDICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INGENIERÍA DE TELECOMUNICACIÓN,Titulacion_is_MÁSTER UNIVERSITARIO TECNOLOGÍAS INFORMACIÓN GREOGRÁFICA SIG Y TELEDETECCI,Titulacion_is_MÁSTER EN SEGURIDAD Y SALUD LABORAL,Titulacion_is_MÁSTER UNIVERSITARIO EN EDUCACIÓN DIGITAL,Titulacion_is_MÁSTER EN ATENCIÓN SOCIO-SANITARIA A PACIENTES CON DEPENDENCIA.,Titulacion_is_MÁSTER EN RECURSOS RENOVABLES E INGENIERÍA ENERGÉTICA,"Titulacion_is_MÁSTER INVESTIGACIÓN ENSEÑANZA Y APRENDIZAJE DE LAS CC. EXPE.,SOCIA. Y MAT",Titulacion_is_MÁSTER UNIVERSITARIO EN CIENCIA Y TECNOLOGÍA DE LA CARNE,Titulacion_is_MÁSTER UNIVERSITARIO ENSEÑANZA DE ESPAÑOL COMO LENGUA EXTRANJERA,Titulacion_is_MÁSTER EN DESARROLLO RURAL,Titulacion_is_MÁSTER EN GESTIÓN DE CALIDAD Y TRAZABILIDAD EN ALIMENTOS DE ORIGEN VEGETAL,Titulacion_is_MÁSTER UNIVERSITARIO EN QUÍMICA SOSTENIBLE,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉ

In [0]:
categoricalCols = ['sexo','tipo_acceso','tipo_erasmus']

In [0]:
dv_personas_190411_model_one_hot = pd.get_dummies(data=dv_personas_190411_model,columns=categoricalCols)

In [6]:
dv_personas_190411_model_one_hot.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,Titulacion_is_MAESTRO: AUDICIÓN Y LENGUAJE,Titulacion_is_LICENCIADO EN DERECHO,Titulacion_is_MAESTRO: EDUCACIÓN PRIMARIA,Titulacion_is_MAESTRO: EDUCACIÓN INFANTIL,Titulacion_is_DIPLOMADO EN TURISMO,"Titulacion_is_INGENIERÍA TÉCNICA EN TELECOMUNICACIONES,ESPECIALIDAD SONIDO E IMAGEN",Titulacion_is_LICENCIADO EN CIENCIA Y TECNOLOGÍA DE LOS ALIMENTOS,Titulacion_is_LICENCIADO EN CIENCIAS AMBIENTALES,Titulacion_is_LICENCIADO EN CIENCIAS DE LA ACTIVIDAD FÍSICA Y DEL DEPORTE,Titulacion_is_LICENCIADO EN FILOLOGÍA HISPÁNICA,Titulacion_is_GRADO EN EDUCACIÓN SOCIAL,Titulacion_is_LICENCIADO EN MATEMÁTICAS,Titulacion_is_LICENCIADO EN FILOLOGÍA FRANCESA,Titulacion_is_LICENCIADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_GRADO EN DERECHO,Titulacion_is_LICENCIADO EN MEDICINA,Titulacion_is_LICENCIADO EN ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_LICENCIADO EN BIOLOGÍA,Titulacion_is_INGENIERO TÉCNICO EN DISEÑO INDUSTRIAL,Titulacion_is_INGENIERO TÉCNICO EN INFORMÁTICA DE SISTEMAS,Titulacion_is_GRADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN RELACIONES LABORALES,Titulacion_is_LICENCIADO EN ANTROPOLOGÍA SOCIAL Y CULTURAL,Titulacion_is_LICENCIADO EN HISTORIA,Titulacion_is_GRADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / ECONOMÍA,Titulacion_is_LICENCIADO EN HUMANIDADES,Titulacion_is_GRADO EN BIOLOGÍA,...,Titulacion_is_MÁSTER EN SEGURIDAD Y SALUD LABORAL,Titulacion_is_MÁSTER UNIVERSITARIO EN EDUCACIÓN DIGITAL,Titulacion_is_MÁSTER EN ATENCIÓN SOCIO-SANITARIA A PACIENTES CON DEPENDENCIA.,Titulacion_is_MÁSTER EN RECURSOS RENOVABLES E INGENIERÍA ENERGÉTICA,"Titulacion_is_MÁSTER INVESTIGACIÓN ENSEÑANZA Y APRENDIZAJE DE LAS CC. EXPE.,SOCIA. Y MAT",Titulacion_is_MÁSTER UNIVERSITARIO EN CIENCIA Y TECNOLOGÍA DE LA CARNE,Titulacion_is_MÁSTER UNIVERSITARIO ENSEÑANZA DE ESPAÑOL COMO LENGUA EXTRANJERA,Titulacion_is_MÁSTER EN DESARROLLO RURAL,Titulacion_is_MÁSTER EN GESTIÓN DE CALIDAD Y TRAZABILIDAD EN ALIMENTOS DE ORIGEN VEGETAL,Titulacion_is_MÁSTER UNIVERSITARIO EN QUÍMICA SOSTENIBLE,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA,sexo_H,sexo_M,tipo_acceso_ACCESO A TERCER CICLO,tipo_acceso_ACCESO A TERCER CICLO CON PERIODO DE DOCENCIA,tipo_acceso_ACCESO A TERCER CICLO CON PERIODO DE INVESTIGACIÓN,tipo_acceso_ACCESO A TERCER CICLO CON SUFICIENCIA INVESTIGADORA,tipo_acceso_ACCESO DOCTORADO (RD 1393/2007),tipo_acceso_ACCESO DOCTORADO (RD 99/2011),tipo_acceso_Acceso a Segundo Ciclo,tipo_acceso_COU sin selectividad,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_eras

In [0]:
dv_personas_190411_model_one_hot.drop(['dni'],axis=1,inplace = True)

#### Procedemos a dividir el tto_pcto_dias_trabajados en bloques de 33

In [8]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,4)
tto_pcto_dias_trabajados_bucket_array

array([  0.        ,  33.33333333,  66.66666667, 100.        ])

In [9]:
dv_personas_190411_model_one_hot_33 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_33.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,Titulacion_is_MAESTRO: AUDICIÓN Y LENGUAJE,Titulacion_is_LICENCIADO EN DERECHO,Titulacion_is_MAESTRO: EDUCACIÓN PRIMARIA,Titulacion_is_MAESTRO: EDUCACIÓN INFANTIL,Titulacion_is_DIPLOMADO EN TURISMO,"Titulacion_is_INGENIERÍA TÉCNICA EN TELECOMUNICACIONES,ESPECIALIDAD SONIDO E IMAGEN",Titulacion_is_LICENCIADO EN CIENCIA Y TECNOLOGÍA DE LOS ALIMENTOS,Titulacion_is_LICENCIADO EN CIENCIAS AMBIENTALES,Titulacion_is_LICENCIADO EN CIENCIAS DE LA ACTIVIDAD FÍSICA Y DEL DEPORTE,Titulacion_is_LICENCIADO EN FILOLOGÍA HISPÁNICA,Titulacion_is_GRADO EN EDUCACIÓN SOCIAL,Titulacion_is_LICENCIADO EN MATEMÁTICAS,Titulacion_is_LICENCIADO EN FILOLOGÍA FRANCESA,Titulacion_is_LICENCIADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_GRADO EN DERECHO,Titulacion_is_LICENCIADO EN MEDICINA,Titulacion_is_LICENCIADO EN ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_LICENCIADO EN BIOLOGÍA,Titulacion_is_INGENIERO TÉCNICO EN DISEÑO INDUSTRIAL,Titulacion_is_INGENIERO TÉCNICO EN INFORMÁTICA DE SISTEMAS,Titulacion_is_GRADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN RELACIONES LABORALES,Titulacion_is_LICENCIADO EN ANTROPOLOGÍA SOCIAL Y CULTURAL,Titulacion_is_LICENCIADO EN HISTORIA,Titulacion_is_GRADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / ECONOMÍA,Titulacion_is_LICENCIADO EN HUMANIDADES,Titulacion_is_GRADO EN BIOLOGÍA,Titulacion_is_INGENIERO TÉCNICO AGRÍCOLA (ESP. EN EXPLOTACIONES AGROPECUARIAS),...,Titulacion_is_MÁSTER EN SEGURIDAD Y SALUD LABORAL,Titulacion_is_MÁSTER UNIVERSITARIO EN EDUCACIÓN DIGITAL,Titulacion_is_MÁSTER EN ATENCIÓN SOCIO-SANITARIA A PACIENTES CON DEPENDENCIA.,Titulacion_is_MÁSTER EN RECURSOS RENOVABLES E INGENIERÍA ENERGÉTICA,"Titulacion_is_MÁSTER INVESTIGACIÓN ENSEÑANZA Y APRENDIZAJE DE LAS CC. EXPE.,SOCIA. Y MAT",Titulacion_is_MÁSTER UNIVERSITARIO EN CIENCIA Y TECNOLOGÍA DE LA CARNE,Titulacion_is_MÁSTER UNIVERSITARIO ENSEÑANZA DE ESPAÑOL COMO LENGUA EXTRANJERA,Titulacion_is_MÁSTER EN DESARROLLO RURAL,Titulacion_is_MÁSTER EN GESTIÓN DE CALIDAD Y TRAZABILIDAD EN ALIMENTOS DE ORIGEN VEGETAL,Titulacion_is_MÁSTER UNIVERSITARIO EN QUÍMICA SOSTENIBLE,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA,sexo_H,sexo_M,tipo_acceso_ACCESO A TERCER CICLO,tipo_acceso_ACCESO A TERCER CICLO CON PERIODO DE DOCENCIA,tipo_acceso_ACCESO A TERCER CICLO CON PERIODO DE INVESTIGACIÓN,tipo_acceso_ACCESO A TERCER CICLO CON SUFICIENCIA INVESTIGADORA,tipo_acceso_ACCESO DOCTORADO (RD 1393/2007),tipo_acceso_ACCESO DOCTORADO (RD 99/2011),tipo_acceso_Acceso a Segundo Ciclo,tipo_acceso_COU sin selectividad,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios 

In [10]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].astype('category')

0        (-0.001, 33.333]
1        (33.333, 66.667]
2        (-0.001, 33.333]
3        (33.333, 66.667]
4        (33.333, 66.667]
               ...       
14710    (-0.001, 33.333]
14711    (-0.001, 33.333]
14712    (-0.001, 33.333]
14713    (-0.001, 33.333]
14714    (-0.001, 33.333]
Name: tto_pcto_dias_trabajados, Length: 14715, dtype: category
Categories (3, interval[float64]): [(-0.001, 33.333] < (33.333, 66.667] < (66.667, 100.0]]

In [11]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_33.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_33.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,Titulacion_is_INGENIERO TÉCNICO EN TOPOGRAFÍA,Titulacion_is_MAESTRO: AUDICIÓN Y LENGUAJE,Titulacion_is_LICENCIADO EN DERECHO,Titulacion_is_MAESTRO: EDUCACIÓN PRIMARIA,Titulacion_is_MAESTRO: EDUCACIÓN INFANTIL,Titulacion_is_DIPLOMADO EN TURISMO,"Titulacion_is_INGENIERÍA TÉCNICA EN TELECOMUNICACIONES,ESPECIALIDAD SONIDO E IMAGEN",Titulacion_is_LICENCIADO EN CIENCIA Y TECNOLOGÍA DE LOS ALIMENTOS,Titulacion_is_LICENCIADO EN CIENCIAS AMBIENTALES,Titulacion_is_LICENCIADO EN CIENCIAS DE LA ACTIVIDAD FÍSICA Y DEL DEPORTE,Titulacion_is_LICENCIADO EN FILOLOGÍA HISPÁNICA,Titulacion_is_GRADO EN EDUCACIÓN SOCIAL,Titulacion_is_LICENCIADO EN MATEMÁTICAS,Titulacion_is_LICENCIADO EN FILOLOGÍA FRANCESA,Titulacion_is_LICENCIADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_GRADO EN DERECHO,Titulacion_is_LICENCIADO EN MEDICINA,Titulacion_is_LICENCIADO EN ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_LICENCIADO EN BIOLOGÍA,Titulacion_is_INGENIERO TÉCNICO EN DISEÑO INDUSTRIAL,Titulacion_is_INGENIERO TÉCNICO EN INFORMÁTICA DE SISTEMAS,Titulacion_is_GRADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN RELACIONES LABORALES,Titulacion_is_LICENCIADO EN ANTROPOLOGÍA SOCIAL Y CULTURAL,Titulacion_is_LICENCIADO EN HISTORIA,Titulacion_is_GRADO EN COMUNICACIÓN AUDIOVISUAL,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / ECONOMÍA,Titulacion_is_LICENCIADO EN HUMANIDADES,Titulacion_is_GRADO EN BIOLOGÍA,Titulacion_is_INGENIERO TÉCNICO AGRÍCOLA (ESP. EN EXPLOTACIONES AGROPECUARIAS),...,Titulacion_is_MÁSTER EN SEGURIDAD Y SALUD LABORAL,Titulacion_is_MÁSTER UNIVERSITARIO EN EDUCACIÓN DIGITAL,Titulacion_is_MÁSTER EN ATENCIÓN SOCIO-SANITARIA A PACIENTES CON DEPENDENCIA.,Titulacion_is_MÁSTER EN RECURSOS RENOVABLES E INGENIERÍA ENERGÉTICA,"Titulacion_is_MÁSTER INVESTIGACIÓN ENSEÑANZA Y APRENDIZAJE DE LAS CC. EXPE.,SOCIA. Y MAT",Titulacion_is_MÁSTER UNIVERSITARIO EN CIENCIA Y TECNOLOGÍA DE LA CARNE,Titulacion_is_MÁSTER UNIVERSITARIO ENSEÑANZA DE ESPAÑOL COMO LENGUA EXTRANJERA,Titulacion_is_MÁSTER EN DESARROLLO RURAL,Titulacion_is_MÁSTER EN GESTIÓN DE CALIDAD Y TRAZABILIDAD EN ALIMENTOS DE ORIGEN VEGETAL,Titulacion_is_MÁSTER UNIVERSITARIO EN QUÍMICA SOSTENIBLE,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA,sexo_H,sexo_M,tipo_acceso_ACCESO A TERCER CICLO,tipo_acceso_ACCESO A TERCER CICLO CON PERIODO DE DOCENCIA,tipo_acceso_ACCESO A TERCER CICLO CON PERIODO DE INVESTIGACIÓN,tipo_acceso_ACCESO A TERCER CICLO CON SUFICIENCIA INVESTIGADORA,tipo_acceso_ACCESO DOCTORADO (RD 1393/2007),tipo_acceso_ACCESO DOCTORADO (RD 99/2011),tipo_acceso_Acceso a Segundo Ciclo,tipo_acceso_COU sin selectividad,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios 

In [12]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].value_counts()

0    9554
1    4121
2    1040
Name: tto_pcto_dias_trabajados, dtype: int64

Como se puede apreciar, hay un gran imbalance en las clases de los datos, la gran mayoría se encuentran en la clase 0 [0,33.33), dado a que las redes nuronales son sensibles a este tipo de casos habrá que aplicar alguna de las técnicas de resolución para este tipo de casos.

In [0]:
Features_1_2 = dv_personas_190411_model_one_hot_33.iloc[:,0:1]
Features_2_2 = dv_personas_190411_model_one_hot_33.iloc[:,2:]
Labels = dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados']

In [14]:
Features = np.array(Features_1_2)
Features = np.concatenate([Features, np.array(Features_2_2)], axis = 1)
Features[:2,:]

array([[23,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0],
       [22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

In [15]:
Features.shape

(14715, 204)

Procedemos a aplicar SMOTETomek que es una solución híbrida entre over-sampling y under-sampling

In [16]:
smt = SMOTETomek()
x_smt, y_smt = smt.fit_sample(Features, Labels)
print(x_smt.shape)
print(y_smt.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(28558, 204)
(28558,)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [17]:
y_smt_0 = y_smt[y_smt == 0]
y_smt_1 = y_smt[y_smt == 0]
y_smt_2 = y_smt[y_smt == 0]
print(y_smt_0.shape)
print(y_smt_1.shape)
print(y_smt_2.shape)

(9504,)
(9504,)
(9504,)


In [0]:
nr.seed(123)
inside = ms.KFold(n_splits=3, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=3, shuffle = True)

In [19]:
param_grid = {"alpha":[0.0000001,0.000001,0.00001], 
              "early_stopping":[True, False], 
              "beta_1":[0.95,0.90,0.80], 
              "beta_2":[0.999,0.9,0.8]}

## Define the Neural Network model
nn_clf = MLPClassifier(hidden_layer_sizes = (100,100),
                       max_iter=300)

## Perform the grid search over the parameters
nr.seed(3456)
nn_clf = ms.GridSearchCV(estimator = nn_clf, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)

nr.seed(6677)
nn_clf.fit(x_smt, y_smt)
print(nn_clf.best_estimator_.alpha)
print(nn_clf.best_estimator_.early_stopping)
print(nn_clf.best_estimator_.beta_1)
print(nn_clf.best_estimator_.beta_2)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1e-06
False
0.8
0.999


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [3]:
nr.seed(498)
cv_estimate = ms.cross_val_score(nn_clf, x_smt, y_smt, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Mean performance metric = 0.797
SDT of the metric       = 0.005
Outcomes by cv fold
Fold  1    0.791
Fold  2    0.800
Fold  3    0.802


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
nr.seed(9988)
indx = range(x_smt.shape[0])
indx = ms.train_test_split(indx, test_size = 3679)
x_train = x_smt[indx[0],:]
y_train = np.ravel(y_smt[indx[0]])
x_test = x_smt[indx[1],:]
y_test = np.ravel(y_smt[indx[1]])

In [4]:
nr.seed(1115)
nn_mod = MLPClassifier(hidden_layer_sizes = (100,100), 
                       alpha = nn_clf.best_estimator_.alpha, 
                       early_stopping = nn_clf.best_estimator_.early_stopping, 
                       beta_1 = nn_clf.best_estimator_.beta_1, 
                       beta_2 = nn_clf.best_estimator_.beta_2,
                       max_iter = 300)
nn_mod.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-06, batch_size='auto', beta_1=0.8,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
scores = nn_mod.predict(x_test)

In [0]:
def print_metrics_33(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]')
    print('Actual [0,333.333]        %6d' % conf[0,0] + '             %5d' % conf[0,1] + '             %5d' % conf[0,2])
    print('Actual (33.333,66.667]    %6d' % conf[1,0] + '             %5d' % conf[1,1] + '             %5d' % conf[1,2])
    print('Actual (66.667,100]       %6d' % conf[2,0] + '             %5d' % conf[2,1] + '             %5d' % conf[2,2])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('         Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]')
    print('Num case   %0.2f' % metrics[3][0] + '     %0.2f' % metrics[3][1] + '      %0.2f' % metrics[3][2] )
    print('Precision   %0.2f' % metrics[0][0] + '      %0.2f' % metrics[0][1] + '       %0.2f' % metrics[0][2] )
    print('Recall      %0.2f' % metrics[1][0] + '      %0.2f' % metrics[1][1] + '       %0.2f' % metrics[1][2] )
    print('F1          %0.2f' % metrics[2][0] + '      %0.2f' % metrics[2][1] + '       %0.2f' % metrics[2][2] )

In [8]:
print_metrics_33(y_test, scores)  

                 Confusion matrix
                 Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]
Actual [0,333.333]           822               279               104
Actual (33.333,66.667]       423               497               301
Actual (66.667,100]          114               132              1007

Accuracy        0.63
 
         Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]
Num case   1205.00     1221.00      1253.00
Precision   0.60      0.55       0.71
Recall      0.68      0.41       0.80
F1          0.64      0.47       0.76


In [0]:
import dill
dill.dump_session('notebook_neuralNetworks.db')

In [0]:
import dill
dill.load_session('notebook_neuralNetworks.db')